In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv_path = "/content/drive/MyDrive/AML_Project_EyeDisease/data/eye_disease_cleaned.csv"
df = pd.read_csv(csv_path)

df.head()


,filepath,label,width,height,label_id,split
0,/content/drive/MyDrive/AML_Project_EyeDisease/...,diabetic_retinopathy,512,512,1,train
1,/content/drive/MyDrive/AML_Project_EyeDisease/...,glaucoma,256,256,2,train
2,/content/drive/MyDrive/AML_Project_EyeDisease/...,normal,512,512,3,train
3,/content/drive/MyDrive/AML_Project_EyeDisease/...,glaucoma,256,256,2,train
4,/content/drive/MyDrive/AML_Project_EyeDisease/...,glaucoma,256,256,2,train


In [ ]:
print(df['split'].value_counts())
print("\nClass distribution:")
print(df['label'].value_counts())


split
train    2951
val       633
test      633
Name: count, dtype: int64

Class distribution:
label
diabetic_retinopathy    1098
normal                  1074
cataract                1038
glaucoma                1007
Name: count, dtype: int64


In [ ]:
train_df = df[df["split"] == "train"].reset_index(drop=True)
val_df   = df[df["split"] == "val"].reset_index(drop=True)
test_df  = df[df["split"] == "test"].reset_index(drop=True)

len(train_df), len(val_df), len(test_df)


(2951, 633, 633)

#Load MobileNetV2 Feature Extractor

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load pretrained MobileNetV2
# include_top = False → remove final classification layer
# pooling = 'avg' → global average pooling (gives 1280-d vector)
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(224, 224, 3)
)

# Freeze the model (we don't train it)
base_model.trainable = False

print("MobileNetV2 model loaded. Output feature size:", base_model.output_shape)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
MobileNetV2 model loaded. Output feature size: (None, 1280)


#Feature Extraction Function

In [ ]:
from tensorflow.keras.preprocessing import image

def extract_features(img_path):
    """
    Loads an image, preprocesses it, and extracts a 1280-d MobileNetV2 feature vector.
    Returns: numpy array of shape (1280,)
    """
    try:
        # Load and resize the image to MobileNetV2 input size
        img = image.load_img(img_path, target_size=(224, 224))

        # Convert image to array
        img_array = image.img_to_array(img)

        # Expand dims → create batch of 1
        img_array = np.expand_dims(img_array, axis=0)

        # Preprocess image (scaling + normalization)
        img_array = preprocess_input(img_array)

        # Extract features (disable verbose output)
        features = base_model.predict(img_array, verbose=0)

        # Flatten to a 1D vector
        return features.flatten()

    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None


In [ ]:
# CELL 7 — Extract features for Train, Val, Test and save them

from tqdm import tqdm
import numpy as np

# Folder where you will save extracted features
save_dir = "/content/drive/MyDrive/AML_Project_EyeDisease/features"
os.makedirs(save_dir, exist_ok=True)

def extract_set_features(df, set_name):
    """
    Extracts features for an entire dataframe (train/val/test)
    and saves them as .npy files.
    """
    feature_list = []
    label_list = []

    print(f"\nExtracting features for {set_name} set...")

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        feat = extract_features(row['filepath'])
        if feat is not None:  # Only append valid features
            feature_list.append(feat)
            label_list.append(row['label_id'])
        else:
            print("Skipped corrupted file:", row['filepath'])

    features = np.array(feature_list)
    labels = np.array(label_list)

    # Save features and labels
    np.save(os.path.join(save_dir, f"{set_name}_features.npy"), features)
    np.save(os.path.join(save_dir, f"{set_name}_labels.npy"), labels)

    print(f"{set_name.capitalize()} set done. Shape: {features.shape}")
    return features, labels


# --- Extract features for each split ---
train_features, train_labels = extract_set_features(train_df, "train")
val_features, val_labels     = extract_set_features(val_df, "val")
test_features, test_labels   = extract_set_features(test_df, "test")



Extracting features for train set...


100%|██████████| 2951/2951 [25:57<00:00,  1.89it/s]


Train set done. Shape: (2951, 1280)

Extracting features for val set...


100%|██████████| 633/633 [05:00<00:00,  2.10it/s]


Val set done. Shape: (633, 1280)

Extracting features for test set...


100%|██████████| 633/633 [05:56<00:00,  1.77it/s]

Test set done. Shape: (633, 1280)


In [1]:
# CELL 8 — PCA for dimensionality reduction (fit on TRAIN only)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import os

# Path where feature files are saved
save_dir = "/content/drive/MyDrive/AML_Project_EyeDisease/features"

# Load extracted features
train_features = np.load(os.path.join(save_dir, "train_features.npy"))
val_features   = np.load(os.path.join(save_dir, "val_features.npy"))
test_features  = np.load(os.path.join(save_dir, "test_features.npy"))

train_labels = np.load(os.path.join(save_dir, "train_labels.npy"))
val_labels   = np.load(os.path.join(save_dir, "val_labels.npy"))
test_labels  = np.load(os.path.join(save_dir, "test_labels.npy"))

print("Loaded feature shapes:")
print("Train:", train_features.shape)
print("Val:", val_features.shape)
print("Test:", test_features.shape)

# -----------------------------
# Standardize features
# -----------------------------
scaler = StandardScaler()

# Fit scaler ONLY on train set
scaler.fit(train_features)

train_scaled = scaler.transform(train_features)
val_scaled   = scaler.transform(val_features)
test_scaled  = scaler.transform(test_features)

print("Scaling done.")

# -----------------------------
# PCA: keep 95% variance
# -----------------------------
pca = PCA(n_components=0.95, random_state=42)

# Fit PCA ONLY on train set
pca.fit(train_scaled)

train_pca = pca.transform(train_scaled)
val_pca   = pca.transform(val_scaled)
test_pca  = pca.transform(test_scaled)

print("PCA complete.")
print("Original dim:", train_features.shape[1])
print("Reduced dim:", train_pca.shape[1])

# -----------------------------
# Save PCA features
# -----------------------------
np.save(os.path.join(save_dir, "train_pca.npy"), train_pca)
np.save(os.path.join(save_dir, "val_pca.npy"), val_pca)
np.save(os.path.join(save_dir, "test_pca.npy"), test_pca)

print("Saved PCA features to:", save_dir)


Loaded feature shapes:
Train: (2951, 1280)
Val: (633, 1280)
Test: (633, 1280)
Scaling done.
PCA complete.
Original dim: 1280
Reduced dim: 572
Saved PCA features to: /content/drive/MyDrive/AML_Project_EyeDisease/features


### model training

In [2]:
# CELL 9 — Logistic Regression on PCA features

import os
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Load PCA features from saved files
save_dir = "/content/drive/MyDrive/AML_Project_EyeDisease/features"

train_pca = np.load(os.path.join(save_dir, "train_pca.npy"))
val_pca   = np.load(os.path.join(save_dir, "val_pca.npy"))
test_pca  = np.load(os.path.join(save_dir, "test_pca.npy"))

train_labels = np.load(os.path.join(save_dir, "train_labels.npy"))
val_labels   = np.load(os.path.join(save_dir, "val_labels.npy"))
test_labels  = np.load(os.path.join(save_dir, "test_labels.npy"))

print("PCA features loaded:")
print("Train:", train_pca.shape)
print("Val:", val_pca.shape)
print("Test:", test_pca.shape)

# ---------------------------------------------
# Train Logistic Regression (multiclass softmax)
# ---------------------------------------------
lr_model = LogisticRegression(
    max_iter=2000,
    multi_class='multinomial',
    solver='lbfgs'
)

lr_model.fit(train_pca, train_labels)

# Predictions
val_pred = lr_model.predict(val_pca)
test_pred = lr_model.predict(test_pca)

# Evaluation
val_acc = accuracy_score(val_labels, val_pred)
test_acc = accuracy_score(test_labels, test_pred)

val_f1 = f1_score(val_labels, val_pred, average='macro')
test_f1 = f1_score(test_labels, test_pred, average='macro')

print("\n=== Logistic Regression Results ===")
print("Validation Accuracy:", val_acc)
print("Validation F1 Score:", val_f1)
print("Test Accuracy:", test_acc)
print("Test F1 Score:", test_f1)

print("\nClassification Report (Test Set):")
print(classification_report(test_labels, test_pred))


PCA features loaded:
Train: (2951, 572)
Val: (633, 572)
Test: (633, 572)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



=== Logistic Regression Results ===
Validation Accuracy: 0.8388625592417062
Validation F1 Score: 0.8381760142917469
Test Accuracy: 0.8593996840442338
Test F1 Score: 0.8573543481360708

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.88      0.89      0.89       156
           1       0.96      0.96      0.96       165
           2       0.80      0.76      0.78       151
           3       0.79      0.81      0.80       161

    accuracy                           0.86       633
   macro avg       0.86      0.86      0.86       633
weighted avg       0.86      0.86      0.86       633



In [4]:
# CELL 10 — SVM (RBF kernel) on PCA features

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

# -------------------------------
# Train SVM with RBF kernel
# -------------------------------
svm_model = SVC(
    kernel='rbf',
    C=5,              # regularization strength
    gamma='scale',    # automatic gamma
    probability=False # no need for probabilities
)

print("Training SVM...")
svm_model.fit(train_pca, train_labels)

# Predictions
val_pred_svm = svm_model.predict(val_pca)
test_pred_svm = svm_model.predict(test_pca)

# Metrics
val_acc_svm = accuracy_score(val_labels, val_pred_svm)
test_acc_svm = accuracy_score(test_labels, test_pred_svm)

val_f1_svm = f1_score(val_labels, val_pred_svm, average='macro')
test_f1_svm = f1_score(test_labels, test_pred_svm, average='macro')

print("\n=== SVM Results (RBF Kernel) ===")
print("Validation Accuracy:", val_acc_svm)
print("Validation F1 Score:", val_f1_svm)
print("Test Accuracy:", test_acc_svm)
print("Test F1 Score:", test_f1_svm)

print("\nClassification Report (Test Set):")
print(classification_report(test_labels, test_pred_svm))


Training SVM...

=== SVM Results (RBF Kernel) ===
Validation Accuracy: 0.9241706161137441
Validation F1 Score: 0.9228879660772737
Test Accuracy: 0.8957345971563981
Test F1 Score: 0.8935321602342907

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       156
           1       0.96      0.98      0.97       165
           2       0.88      0.78      0.83       151
           3       0.84      0.90      0.87       161

    accuracy                           0.90       633
   macro avg       0.90      0.89      0.89       633
weighted avg       0.90      0.90      0.89       633



In [5]:
# CELL 11 — Random Forest on PCA features

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# -------------------------------
# Train Random Forest
# -------------------------------
rf_model = RandomForestClassifier(
    n_estimators=200,   # number of trees
    max_depth=None,     # allow deep trees
    random_state=42,
    n_jobs=-1           # use all CPU cores
)

print("Training Random Forest...")
rf_model.fit(train_pca, train_labels)

# Predictions
val_pred_rf = rf_model.predict(val_pca)
test_pred_rf = rf_model.predict(test_pca)

# Metrics
val_acc_rf = accuracy_score(val_labels, val_pred_rf)
test_acc_rf = accuracy_score(test_labels, test_pred_rf)

val_f1_rf = f1_score(val_labels, val_pred_rf, average='macro')
test_f1_rf = f1_score(test_labels, test_pred_rf, average='macro')

print("\n=== Random Forest Results ===")
print("Validation Accuracy:", val_acc_rf)
print("Validation F1 Score:", val_f1_rf)
print("Test Accuracy:", test_acc_rf)
print("Test F1 Score:", test_f1_rf)

print("\nClassification Report (Test Set):")
print(classification_report(test_labels, test_pred_rf))


Training Random Forest...

=== Random Forest Results ===
Validation Accuracy: 0.8151658767772512
Validation F1 Score: 0.816267858590457
Test Accuracy: 0.8088467614533965
Test F1 Score: 0.8081073711115835

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.87      0.85      0.86       156
           1       0.85      0.83      0.84       165
           2       0.82      0.70      0.75       151
           3       0.72      0.85      0.78       161

    accuracy                           0.81       633
   macro avg       0.81      0.81      0.81       633
weighted avg       0.81      0.81      0.81       633



In [6]:
# CELL 12 — MLP Neural Network (Multilayer Perceptron)

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# -------------------------------
# Train MLP Neural Network
# -------------------------------
mlp_model = MLPClassifier(
    hidden_layer_sizes=(256, 128),  # two hidden layers
    activation='relu',
    solver='adam',
    max_iter=300,
    random_state=42
)

print("Training MLP Neural Network...")
mlp_model.fit(train_pca, train_labels)

# Predictions
val_pred_mlp = mlp_model.predict(val_pca)
test_pred_mlp = mlp_model.predict(test_pca)

# Metrics
val_acc_mlp = accuracy_score(val_labels, val_pred_mlp)
test_acc_mlp = accuracy_score(test_labels, test_pred_mlp)

val_f1_mlp = f1_score(val_labels, val_pred_mlp, average='macro')
test_f1_mlp = f1_score(test_labels, test_pred_mlp, average='macro')

print("\n=== MLP Neural Network Results ===")
print("Validation Accuracy:", val_acc_mlp)
print("Validation F1 Score:", val_f1_mlp)
print("Test Accuracy:", test_acc_mlp)
print("Test F1 Score:", test_f1_mlp)

print("\nClassification Report (Test Set):")
print(classification_report(test_labels, test_pred_mlp))


Training MLP Neural Network...

=== MLP Neural Network Results ===
Validation Accuracy: 0.9036334913112164
Validation F1 Score: 0.9023812674189775
Test Accuracy: 0.8925750394944708
Test F1 Score: 0.8910025874108067

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.87      0.90      0.89       156
           1       0.99      0.96      0.97       165
           2       0.89      0.79      0.84       151
           3       0.83      0.91      0.87       161

    accuracy                           0.89       633
   macro avg       0.89      0.89      0.89       633
weighted avg       0.89      0.89      0.89       633



In [8]:
# CELL 13 — XGBoost Classifier (NEW METHOD)

!pip install xgboost

import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Convert labels to int for XGBoost
train_labels_int = train_labels.astype(int)
val_labels_int = val_labels.astype(int)
test_labels_int = test_labels.astype(int)

# ---------------------------------------
# Train XGBoost (simple but effective)
# ---------------------------------------
xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softmax',
    num_class=len(np.unique(train_labels)),
    eval_metric='mlogloss',
    random_state=42
)

print("Training XGBoost...")
xgb_model.fit(train_pca, train_labels_int)

# Predictions
val_pred_xgb = xgb_model.predict(val_pca)
test_pred_xgb = xgb_model.predict(test_pca)

# Metrics
val_acc_xgb = accuracy_score(val_labels_int, val_pred_xgb)
test_acc_xgb = accuracy_score(test_labels_int, test_pred_xgb)

val_f1_xgb = f1_score(val_labels_int, val_pred_xgb, average='macro')
test_f1_xgb = f1_score(test_labels_int, test_pred_xgb, average='macro')

print("\n=== XGBoost Results (NEW METHOD) ===")
print("Validation Accuracy:", val_acc_xgb)
print("Validation F1 Score:", val_f1_xgb)
print("Test Accuracy:", test_acc_xgb)
print("Test F1 Score:", test_f1_xgb)

print("\nClassification Report (Test Set):")
print(classification_report(test_labels_int, test_pred_xgb))


Training XGBoost...

=== XGBoost Results (NEW METHOD) ===
Validation Accuracy: 0.8625592417061612
Validation F1 Score: 0.8616233121046366
Test Accuracy: 0.8593996840442338
Test F1 Score: 0.8580990877842546

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.89      0.89      0.89       156
           1       0.94      0.91      0.93       165
           2       0.83      0.75      0.79       151
           3       0.78      0.88      0.83       161

    accuracy                           0.86       633
   macro avg       0.86      0.86      0.86       633
weighted avg       0.86      0.86      0.86       633



In [9]:
# CELL 14 — Comparison Table for All Models

import pandas as pd

# Collect results for each model
results = {
    "Model": [
        "Logistic Regression",
        "SVM (RBF)",
        "Random Forest",
        "MLP Neural Network",
        "XGBoost (New Method)"
    ],
    "Val Accuracy": [
        val_acc,
        val_acc_svm,
        val_acc_rf,
        val_acc_mlp,
        val_acc_xgb
    ],
    "Test Accuracy": [
        test_acc,
        test_acc_svm,
        test_acc_rf,
        test_acc_mlp,
        test_acc_xgb
    ],
    "Val F1 (Macro)": [
        val_f1,
        val_f1_svm,
        val_f1_rf,
        val_f1_mlp,
        val_f1_xgb
    ],
    "Test F1 (Macro)": [
        test_f1,
        test_f1_svm,
        test_f1_rf,
        test_f1_mlp,
        test_f1_xgb
    ]
}

# Create DataFrame
results_df = pd.DataFrame(results)

# Display table nicely
print("\n=== MODEL PERFORMANCE SUMMARY ===")
results_df



=== MODEL PERFORMANCE SUMMARY ===


,Model,Val Accuracy,Test Accuracy,Val F1 (Macro),Test F1 (Macro)
0,Logistic Regression,0.838863,0.859400,0.838176,0.857354
1,SVM (RBF),0.924171,0.895735,0.922888,0.893532
2,Random Forest,0.815166,0.808847,0.816268,0.808107
3,MLP Neural Network,0.903633,0.892575,0.902381,0.891003
4,XGBoost (New Method),0.862559,0.859400,0.861623,0.858099
